# Cleaning the data
The following will be done to clean the data we have scraped
- Deal with any missing data DONE
- Remove all the numbers and square brackets in the values in the columns DONE
- `url` on all the csvs will be dropped DONE
- Remove the extra \n words in the names DONE
- Change the naming of some columns to have a uniform naming DONE
- Combine several columns to make a character dataframe 
- Check for null entries
- Check for duplicated rows
- Check for the data types
- Check for spelling errors 

In [2]:
from IPython.display import display
import pandas as pd
import sqlite3
import os
import re

In [3]:
# Read data from the csvs
data_dir = 'data'
dfs = {}
for filename in os.listdir(data_dir):
    if filename.endswith('.csv'):
        name = os.path.splitext(filename)[0]
        path = os.path.join(data_dir, filename) 
        dfs[name] = pd.read_csv(path)


In [4]:
class DataClean:
    # General info of the dataframes
    def __init__(self,df:dict):
        self.df = df
        
    # Check for missing data
    def missing_data(self):
        missing = []
        for col_name, df_data in self.df.items():
            missing_percent = round(df_data.isna().sum() / len(df_data) * 100, 2)
            missing_percents = pd.DataFrame({'column_name': col_name, 'percent_missing': missing_percent})
            missing_percents.drop(missing_percents[missing_percents["percent_missing"] == 0].index, inplace=True)
            missing.append(missing_percents)
        display(pd.concat(missing))

    # Replace missing data with unknown
    def replace_missing(self):
        for col_name, df_data in self.df.items():
            df_data.fillna('Unknown', inplace=True)

    # Remove unwanted numbers and parentheses from the dataframe
    def rmv_extras(self):
        for df_name, df_data in self.df.items():
            for col_name in df_data.columns:
                df_data[col_name] = df_data[col_name].map(lambda x : re.sub(pattern='\d|\(.*|[[\]]|\s|[\"]',repl=' ',string=x).replace('Ghost','').strip())
    
    # Drop the url column from the dataframes
    def drop_col(self, col='url'):
            for df_name, df_data in self.df.items():
                if col in df_data.columns:
                        df_data.drop(col, axis=1, inplace=True)
                else:
                    print(f"{col} column does not exist in {df_name}")
    
    # Rename some columns
    def rename_cols(self, cols:dict):
        for df_name, df_data in self.df.items():
            df_data.rename(columns=cols, inplace=True)
                

In [5]:
# Initialize the cDataClean class
data_clean = DataClean(dfs)

In [6]:
# Check for missing values
data_clean.missing_data()

,column_name,percent_missing
hair colour,enemies_df,33.33
eye colour,enemies_df,77.78
skin colour,enemies_df,77.78
url,house_df,2.86
species,house_df,5.71
gender,house_df,2.86
hair colour,house_df,17.14
eye colour,house_df,25.71
skin colour,house_df,34.29
species,others_df,23.26


In [7]:
# Deal with the missing data by changing all the values of the Nans to Unknown 
data_clean.replace_missing()

In [8]:
# Confirm if the missing data has been removed
data_clean.missing_data()

,column_name,percent_missing


In [9]:
# Remove any unwanted values in the dataframe
data_clean.rmv_extras()

C:\Users\meshw\AppData\Local\Temp\ipykernel_7688\4007493487.py:25: FutureWarning: Possible nested set at position 9
  df_data[col_name] = df_data[col_name].map(lambda x : re.sub(pattern='\d|\(.*|[[\]]|\s|[\"]',repl=' ',string=x).replace('Ghost','').strip())


In [10]:
# Drop url column
data_clean.drop_col()

url column does not exist in creatures_df
url column does not exist in locations_df


In [11]:
# Confirm that the url column has been dropped
data_clean.drop_col()

url column does not exist in creatures_df
url column does not exist in enemies_df
url column does not exist in house_df
url column does not exist in locations_df
url column does not exist in others_df
url column does not exist in spells_df
url column does not exist in staff_df
url column does not exist in villagers_df
url column does not exist in wizards_df


In [12]:
dfs['staff_df']

,name,species,gender,hair colour,eye colour,skin colour
0,Phineas Nigellus Black,Human,Male,Black,Blue,Pale
1,Matilda Weasley,Unknown,Unknown,Unknown,Unknown,Unknown
2,Eleazar Fig,Human,Male,Grey,Unknown,Unknown
3,Aesop Sharp,Human,Male,Brown,Unknown,Unknown
4,Dinah Hecat,Human,Female,Grey,Brown,Light
5,Mirabel Garlick,Human,Female,Red,Unknown,Unknown
6,Abraham Ronen,Human,Male,',Grey,Brown
7,Cuthbert Binns,Human,Male,White,Black,Pale
8,Bai Howin,Human,Female,Black,Brown,Light
9,Chiyo Kogawa,Human,Female,Black,Unknown,Unknown


In [73]:
# Rename columns for uniformity
cols = {
    'creature_type': 'species', 
    'creatures' : 'name',
    'hair colour': 'hair_colour',
    'eye colour': 'eye_colour',
    'skin colour': 'skin_colour',
    'hand movement': 'hand_movement'
}
data_clean.rename_cols(cols)

In [74]:
# Save the cleaned data into csv format
data_dir = 'cleaned_data'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

for name, df in dfs.items():
    filename = os.path.join(data_dir, f'{name}.csv')
    df.to_csv(filename, index_label=False)

In [16]:
conn = sqlite3.connect('hogwarts_legact.db')
cur = conn.cursor()

In [1]:
dfs['staff_df']

NameError: name 'dfs' is not defined